# Filtrado

In [7]:
import numpy as np
import netCDF4 as nc4
from netCDF4 import num2date
import pandas as pd
from datetime import datetime

In [ ]:
# Ruta del archivo .nc
ncFileName = '/disk/barbusano/barbusano1/Projects/Waves/IAwave/Tania_TFG/Datos/Datos_Albano_Modelo/wave_wind_2000_2023_128x128.nc'

with nc4.Dataset(ncFileName, mode='r') as ncfid:
    # Extraemos variables
    time = ncfid.variables['valid_time'][:]
    lat = ncfid.variables['latitude'][:]
    lon = ncfid.variables['longitude'][:]
    u10 = ncfid.variables['u10'][:]
    v10 = ncfid.variables['v10'][:]

    # Convertimos tiempos a fechas utilizando netCDF4.num2date
    time_units = ncfid.variables['valid_time'].units
    time_calendar = ncfid.variables['valid_time'].calendar if hasattr(ncfid.variables['valid_time'], 'calendar') else 'standard'

    # Convertimos cftime.DatetimeGregorian a datetime.datetime objects
    time_dates = num2date(time, units=time_units, calendar=time_calendar)
    time_dates_dt = [datetime(date.year, date.month, date.day) for date in time_dates]


# Verifiquemos las dimensiones para ver que son las mismas de entrada y de salida, ya que en las CNN
# los datos van en pares:
print("Dimensiones de las variables:")
print(f"time: {time.shape}, lat: {lat.shape}, lon: {lon.shape}")
print(f"u10: {u10.shape}, v10: {v10.shape}")

# Veamos porque normalizar esto va a estar curioso
print(f"Valores mínimos y máximos de u10: {np.min(u10)}, {np.max(u10)}")
print(f"Valores mínimos y máximos de v10: {np.min(v10)}, {np.max(v10)}")

Dimensiones de las variables:
time: (7974,), lat: (128,), lon: (128,)
u10: (7974, 128, 128), v10: (7974, 128, 128)
Valores mínimos y máximos de u10: -23.576358795166016, 26.41472053527832
Valores mínimos y máximos de v10: -25.095306396484375, 22.78137969970703


In [9]:
# Ruta del archivo CSV
csv_filepath = "/disk/barbusano/barbusano1/Projects/Waves/IAwave/Tania_TFG/Datos/Boya Tenerife Sur/MD_DailyWaveStats_4p_20_23_TS.csv"  # Cambia esto a la ruta de tu archivo .csv

# Leemos el .csv directamente desde la ruta especificada
df = pd.read_csv(csv_filepath, sep=';', decimal=',', encoding='latin-1')

# Mostrar las primeras filas del DataFrame
print(df.head())

# Obtener información sobre las columnas y tipos de datos
print(df.info())

  Fechas Faltantes
0       2000-01-01
1       2000-01-02
2       2000-01-03
3       2000-01-04
4       2000-01-05
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Fechas Faltantes  1887 non-null   object
dtypes: object(1)
memory usage: 14.9+ KB
None


In [10]:
df['Fechas Faltantes'] = pd.to_datetime(df['Fechas Faltantes'], errors='coerce')
print(df.head())  # Para revisar las primeras filas
print(df.info())  # Para verificar el tipo de dato

  Fechas Faltantes
0       2000-01-01
1       2000-01-02
2       2000-01-03
3       2000-01-04
4       2000-01-05
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Fechas Faltantes  1887 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 14.9 KB
None


In [11]:
# Convertimos a standard objects y a pd.time
time_dates = [datetime(t.year, t.month, t.day, t.hour, t.minute, t.second) for t in time_dates]
time_dates = pd.to_datetime(time_dates)

# Cargar las fechas que queremos eliminar
missing_dates_df = pd.read_csv('/disk/barbusano/barbusano1/Projects/Waves/IAwave/Tania_TFG/Datos/Boya Tenerife Sur/MD_DailyWaveStats_4p_20_23_TS.csv', sep=';', decimal=',', encoding='latin-1')

# Nos aseguramos de que el formato de las missing es datetime
missing_dates_df['Fechas Faltantes'] = pd.to_datetime(missing_dates_df['Fechas Faltantes'])

# Convert the column to DatetimeIndex and update the DataFrame
missing_dates = pd.DatetimeIndex(missing_dates_df['Fechas Faltantes']) # Asegurar formato DatetimeIndex

# Quitamos las horas porque si no no coincidiran wey
time_dates = time_dates.floor('D')
missing_dates = missing_dates.floor('D')

In [12]:
# Creamos una máscara para eliminar solo las fechas contenidas en missing_dates
mask = ~time_dates.isin(missing_dates)

# Aplicamos la máscara a todas las variables
time_filtered = time[mask]
u10_filtered = u10[mask]
v10_filtered = v10[mask]

print(f"Fechas originales: {len(time)}, Fechas después de filtrar: {len(time_filtered)}")


Fechas originales: 7974, Fechas después de filtrar: 6879


In [13]:
# Ver si tienen el mismo formato
print("Ejemplo de fechas en time_dates:", time_dates[:5])
print("Ejemplo de fechas en missing_dates:", missing_dates[:5])

Ejemplo de fechas en time_dates: DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05'],
              dtype='datetime64[ns]', freq=None)
Ejemplo de fechas en missing_dates: DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05'],
              dtype='datetime64[ns]', name='Fechas Faltantes', freq=None)


In [14]:
import numpy as np
import netCDF4 as nc4
import cftime

# Ruta del archivo donde se guardará el nuevo .nc
nuevo_nc_filename = "/disk/barbusano/barbusano1/Projects/Waves/IAwave/Tania_TFG/Datos/Datos_Albano_Modelo/wave_wind_20_23_filtered_3y.nc"

# Crear el nuevo archivo .nc
with nc4.Dataset(nuevo_nc_filename, "w", format="NETCDF3_CLASSIC") as new_nc:
    # Crear dimensiones
    new_nc.createDimension("time", len(time_filtered))
    new_nc.createDimension("lat", len(lat))
    new_nc.createDimension("lon", len(lon))

    # Crear las variables
    times_var = new_nc.createVariable("valid_time", "f8", ("time",))
    lat_var = new_nc.createVariable("latitude", "f4", ("lat",))
    lon_var = new_nc.createVariable("longitude", "f4", ("lon",))
    u10_var = new_nc.createVariable("u10", "f4", ("time", "lat", "lon"))
    v10_var = new_nc.createVariable("v10", "f4", ("time", "lat", "lon"))

    # Asignar atributos de tiempo
    times_var.units = time_units
    times_var.calendar = time_calendar

    # Verificar el tipo de time_filtered antes de asignarlo
    if isinstance(time_filtered[0], (np.datetime64, cftime.datetime)):
        # Si es datetime, convertirlo a número
        time_filtered_numeric = nc4.date2num(time_filtered, units=time_units, calendar=time_calendar)
    else:
        # Si ya es numérico, usarlo directamente
        time_filtered_numeric = time_filtered

    # Escribir los datos filtrados
    times_var[:] = time_filtered_numeric
    lat_var[:] = lat  # Las latitudes no cambian
    lon_var[:] = lon  # Las longitudes no cambian
    u10_var[:] = u10_filtered
    v10_var[:] = v10_filtered

    # Forzar la escritura a disco
    new_nc.sync()

print(f"Archivo filtrado guardado en: {nuevo_nc_filename}")


Archivo filtrado guardado en: /disk/barbusano/barbusano1/Projects/Waves/IAwave/Tania_TFG/Datos/Datos_Albano_Modelo/wave_wind_20_23_filtered_3y.nc
